In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
#Read Churn Dataset and get summary of basic statistics
df=pd.read_csv('./Resources/a_bin_5_df.csv')
df.describe()

,Unnamed: 0,CreditScore,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,bin_age,gender,Geography_France,Geography_Germany,Geography_Spain
count,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000
mean,4996.449682,650.287830,5.013974,76552.236761,1.530921,0.705809,0.502138,100200.278401,0.202628,4.687246,0.546042,0.501408,0.251434,0.247158
std,2888.503018,96.660575,2.888560,62422.133272,0.581498,0.455702,0.500022,57500.956105,0.401979,2.082193,0.497902,0.500024,0.433860,0.431382
min,0.000000,350.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000
25%,2486.000000,583.000000,3.000000,0.000000,1.000000,0.000000,0.000000,51226.320000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000
50%,5005.000000,652.000000,5.000000,97266.100000,1.000000,1.000000,1.000000,100335.550000,0.000000,5.000000,1.000000,1.000000,0.000000,0.000000
75%,7493.000000,717.000000,7.000000,127661.690000,2.000000,1.000000,1.000000,149458.730000,0.000000,6.000000,1.000000,1.000000,1.000000,0.000000
max,9999.000000,850.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000,10.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9589 entries, 0 to 9588
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         9589 non-null   int64  
 1   CreditScore        9589 non-null   int64  
 2   Tenure             9589 non-null   int64  
 3   Balance            9589 non-null   float64
 4   NumOfProducts      9589 non-null   int64  
 5   HasCrCard          9589 non-null   int64  
 6   IsActiveMember     9589 non-null   int64  
 7   EstimatedSalary    9589 non-null   float64
 8   Exited             9589 non-null   int64  
 9   bin_age            9589 non-null   int64  
 10  gender             9589 non-null   int64  
 11  Geography_France   9589 non-null   int64  
 12  Geography_Germany  9589 non-null   int64  
 13  Geography_Spain    9589 non-null   int64  
dtypes: float64(2), int64(12)
memory usage: 1.0 MB


### Smoteenn Remove Age Outliers

In [5]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

In [6]:
from pathlib import Path
from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [7]:
#Create Features
X = df.drop(["Exited"], axis=1)

# Create our target
y = df["Exited"]

In [8]:
# Check the balance of our target values
y.value_counts()

0    7646
1    1943
Name: Exited, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
Counter(y_train)

Counter({0: 5740, 1: 1451})

In [10]:
# Standardize the data with StandardScaler().
scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
# Combined over and under sampling for imbalanced classification
from imblearn.combine import SMOTEENN

# define sampling
comb_sample = SMOTEENN(random_state=42)
X_resample, y_resample = comb_sample.fit_resample(X_train, y_train)

# Check if SMOTE-ENN were properly applied
Counter(y_resample)

Counter({0: 3433, 1: 4894})

### Fit Different Module Define

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

### AdaBoost Model with SMOTEENN

In [13]:
# Initiate the model
smoteenn_adb = AdaBoostClassifier()

# Fit the model
smoteenn_adb_model = smoteenn_adb.fit(X_resample, y_resample.ravel())

# Make Predictions
y_pred = smoteenn_adb_model.predict(X_test)

In [14]:
# Calculated the balanced accuracy score
smoteenn_adb_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_adb_accuracy

0.7043887936256068

In [15]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1275,631
Actual Exited,128,364


In [16]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn adb classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn adb classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.91      0.67      0.74      0.77      0.70      0.49      1906
          1       0.37      0.74      0.67      0.49      0.70      0.50       492

avg / total       0.80      0.68      0.73      0.71      0.70      0.49      2398



### KNN with SMOTEENN

In [17]:
# Initiate the model
smoteenn_knn = KNeighborsClassifier()

# Fit the model
smoteenn_knn_model = smoteenn_knn.fit(X_resample, y_resample.ravel())

# Make Predictions
y_pred=smoteenn_knn_model.predict(X_test)

In [18]:
# Calculated the balanced accuracy score
smoteenn_knn_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_knn_accuracy

0.6627509192195804

In [19]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1217,689
Actual Exited,154,338


In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn knn classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn knn classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.89      0.64      0.69      0.74      0.66      0.44      1906
          1       0.33      0.69      0.64      0.45      0.66      0.44       492

avg / total       0.77      0.65      0.68      0.68      0.66      0.44      2398



### SVM with SMOTEENN

In [21]:
# Initiate the model
smoteenn_svc = SVC(kernel='rbf',probability=True)

# Fit the model
smoteenn_svc_model = smoteenn_svc.fit(X_resample, y_resample.ravel())

# Make Predictions
y_pred=smoteenn_svc_model.predict(X_test)

In [22]:
# Calculated the balanced accuracy score
smoteenn_svc_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_svc_accuracy

0.6900225219461009

In [23]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1166,740
Actual Exited,114,378


In [24]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn svc classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn svc classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.91      0.61      0.77      0.73      0.69      0.46      1906
          1       0.34      0.77      0.61      0.47      0.69      0.48       492

avg / total       0.79      0.64      0.74      0.68      0.69      0.47      2398



### Decision Tree with SMOTEENN

In [25]:
# Initiate the model
smoteenn_tree = DecisionTreeClassifier()

# Fit the model
smoteenn_tree_model = smoteenn_tree.fit(X_resample, y_resample.ravel())

# Make Predictions
y_pred=smoteenn_tree_model.predict(X_test)

In [26]:
# Calculated the balanced accuracy score
smoteenn_tree_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_tree_accuracy

0.6854168266236702

In [27]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1408,498
Actual Exited,181,311


In [28]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn tree classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn tree classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.89      0.74      0.63      0.81      0.68      0.47      1906
          1       0.38      0.63      0.74      0.48      0.68      0.46       492

avg / total       0.78      0.72      0.65      0.74      0.68      0.47      2398



### Random Forest with SMOTEENN

In [29]:
# Initiate the model
smoteenn_rfc = RandomForestClassifier()

# Fit the model
smoteenn_rfc_model = smoteenn_rfc.fit(X_resample, y_resample.ravel())

# Make Predictions
y_pred = smoteenn_rfc_model.predict(X_test)

In [30]:
# Calculated the balanced accuracy score
smoteenn_rfc_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_rfc_accuracy

0.7100384749912556

In [31]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1436,470
Actual Exited,164,328


In [32]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn rfc classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn rfc classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.90      0.75      0.67      0.82      0.71      0.51      1906
          1       0.41      0.67      0.75      0.51      0.71      0.50       492

avg / total       0.80      0.74      0.68      0.76      0.71      0.50      2398



### XGBoost Classifier with SMOTEENN

In [33]:
# Initiate the model
smoteenn_xgb = XGBClassifier()
# Fit the model
smoteenn_xgb_model =smoteenn_xgb.fit(X_resample, y_resample.ravel())
# Make Predictions
y_pred = smoteenn_xgb_model.predict(X_test)

In [34]:
# Calculated the balanced accuracy score
smoteenn_xgb_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_xgb_accuracy

0.7069299772221227

In [35]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1521,385
Actual Exited,189,303


In [36]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn xgb classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn xgb classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.89      0.80      0.62      0.84      0.70      0.50      1906
          1       0.44      0.62      0.80      0.51      0.70      0.48       492

avg / total       0.80      0.76      0.65      0.77      0.70      0.50      2398



### Logistic Regression with SMOTEENN

In [37]:
# Initiate the model
smoteenn_lm = LogisticRegression()

# Fit the model
smoteenn_lm_model = smoteenn_lm.fit(X_resample, y_resample.ravel())

# Make Predictions
y_pred=smoteenn_lm_model.predict(X_test)

In [38]:
# Calculated the balanced accuracy score
smoteenn_lm_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_lm_accuracy

0.5984386063692746

In [39]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,844,1062
Actual Exited,121,371


In [40]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn lm classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn lm classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.87      0.44      0.75      0.59      0.58      0.32      1906
          1       0.26      0.75      0.44      0.39      0.58      0.34       492

avg / total       0.75      0.51      0.69      0.55      0.58      0.33      2398

